In [17]:
#glossdoor
import pandas as pd
from geopy.geocoders import Nominatim
geoloc = Nominatim(user_agent='Pruebas')
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geoloc.geocode, min_delay_seconds=3)
from cachetools import LRUCache
from geopy.exc import GeocoderServiceError
from tqdm import tqdm
import time


In [2]:
df = pd.read_csv(r'Dataset\glassdoor.csv') 

In [24]:

#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [3]:

df[df['map.country'].isna()].isna().sum()

benefits.benefitRatingDecimal           0
benefits.comments                       1
benefits.highlights                     1
benefits.numRatings                     0
benefits.employerSummary            46607
                                    ...  
salary.currency.positiveTemplate    15867
salary.currency.symbol              15867
salary.lastSalaryDate               15867
salary.salaries                       742
wwfu                                40749
Length: 163, dtype: int64

In [4]:
recuperar = df[df['map.country'].isna()] 

#Estos son los posibles datos que podriamos recuperar, ya que aunque no tengamos los 
#paises podemos por ejemplo usar una libreria(geopy) para saber el codigo pais a partir
#de la latitud y longitud 


In [5]:
recuperar.head()

,benefits.benefitRatingDecimal,benefits.comments,benefits.highlights,benefits.numRatings,benefits.employerSummary,breadCrumbs,gaTrackerData.category,gaTrackerData.empId,gaTrackerData.empName,gaTrackerData.empSize,...,salary.currency.displayName,salary.currency.id,salary.currency.name,salary.currency.negativeTemplate,salary.currency.new,salary.currency.positiveTemplate,salary.currency.symbol,salary.lastSalaryDate,salary.salaries,wwfu
1,4.000000,2802.0,2802.0,3,NaN,2802,-1,10416,Boehringer Ingelheim,10000--1,...,United Kingdom Pound (GBP),2.0,United Kingdom Pound,$(#),False,$#,&pound;,2019-10-14T06:05:58,2484.0,NaN
5,3.000000,2806.0,2806.0,20,NaN,2806,-1,4212,NCR,10000--1,...,United Kingdom Pound (GBP),2.0,United Kingdom Pound,$(#),False,$#,&pound;,2019-10-12T11:57:27,2488.0,NaN
19,0.000000,2820.0,2820.0,0,NaN,2820,-1,264397,Beumer,1001-5000,...,United Kingdom Pound (GBP),2.0,United Kingdom Pound,$(#),False,$#,&pound;,2019-08-30T02:46:27,2501.0,NaN
22,3.333333,2823.0,2823.0,3,NaN,2823,10007,97,Best Buy,10000--1,...,United Kingdom Pound (GBP),2.0,United Kingdom Pound,$(#),False,$#,&pound;,2019-10-30T03:56:59,2503.0,NaN
23,0.000000,2824.0,2824.0,0,NaN,2824,-1,428690,Schrödinger,201-500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2504.0,NaN


In [6]:
#Aquellas filas que tengan la latitud y longitud igual a 0, los podemos 
#considerar nulos ya que no hay nada en esas cordenadas
contador = sum((recuperar['map.lat'] == 0) & (recuperar['map.lng'] == 0))

In [7]:
contador

5710

In [8]:
filtro = (recuperar['map.lat'] == 0) & (recuperar['map.lng'] == 0)
recuperar_filt = recuperar[~filtro] 

In [10]:
recuperar_filt
#Bueno de esta manera reduciomos los datos con los que trabajar lo mas que 
#pudimos, esto se debe a que el codigo a usar no es precisamente rapido, 
#aun asi son muchas las filas con las que trabajamos

,benefits.benefitRatingDecimal,benefits.comments,benefits.highlights,benefits.numRatings,benefits.employerSummary,breadCrumbs,gaTrackerData.category,gaTrackerData.empId,gaTrackerData.empName,gaTrackerData.empSize,...,salary.currency.displayName,salary.currency.id,salary.currency.name,salary.currency.negativeTemplate,salary.currency.new,salary.currency.positiveTemplate,salary.currency.symbol,salary.lastSalaryDate,salary.salaries,wwfu
1,4.000000,2802.0,2802.0,3,NaN,2802,-1,10416,Boehringer Ingelheim,10000--1,...,United Kingdom Pound (GBP),2.0,United Kingdom Pound,$(#),False,$#,&pound;,2019-10-14T06:05:58,2484.0,NaN
19,0.000000,2820.0,2820.0,0,NaN,2820,-1,264397,Beumer,1001-5000,...,United Kingdom Pound (GBP),2.0,United Kingdom Pound,$(#),False,$#,&pound;,2019-08-30T02:46:27,2501.0,NaN
22,3.333333,2823.0,2823.0,3,NaN,2823,10007,97,Best Buy,10000--1,...,United Kingdom Pound (GBP),2.0,United Kingdom Pound,$(#),False,$#,&pound;,2019-10-30T03:56:59,2503.0,NaN
23,0.000000,2824.0,2824.0,0,NaN,2824,-1,428690,Schrödinger,201-500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2504.0,NaN
24,3.571429,2825.0,2825.0,36,<p><strong>Your Learning &amp; Reward…</strong...,2825,-1,3363,Samsung Electronics,10000--1,...,United Kingdom Pound (GBP),2.0,United Kingdom Pound,$(#),False,$#,&pound;,2019-10-14T01:40:24,2505.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165271,3.857143,58382.0,58382.0,7,NaN,58382,-1,122537,Guidewire,1001-5000,...,United Kingdom Pound (GBP),2.0,United Kingdom Pound,$(#),False,$#,&pound;,2019-01-04T04:21:36,52064.0,NaN
165273,0.000000,58384.0,58384.0,0,NaN,58384,-1,1275455,Acción contra el Hambre,5001-10000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52066.0,NaN
165274,3.666667,58385.0,58385.0,3,NaN,58385,-1,1067,Western Digital,10000--1,...,United Kingdom Pound (GBP),2.0,United Kingdom Pound,$(#),False,$#,&pound;,2019-10-28T09:28:42,52067.0,NaN
165275,3.695652,58386.0,58386.0,23,Publicis Sapient understands the value of prov...,58386,-1,1646026,Publicis Sapient,10000--1,...,United Kingdom Pound (GBP),2.0,United Kingdom Pound,$(#),False,$#,&pound;,2019-10-24T08:17:03,52068.0,6615.0


In [11]:
cord = recuperar_filt[['map.lat','map.lng']]
cord.head()

,map.lat,map.lng
1,41.1500,-8.6167
19,1.2931,103.8558
22,42.3708,-71.1833
23,35.6850,139.7514
24,-22.9000,-47.0833


In [12]:

for i, row in cord.iterrows():
    cord.loc[i, 'Cordenada'] = str(row['map.lat']) + ', ' + str(row['map.lng'])
    


C:\Users\Usuario\AppData\Local\Temp\ipykernel_18200\658970922.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cord.loc[i, 'Cordenada'] = str(row['map.lat']) + ', ' + str(row['map.lng'])


In [13]:
cord
#Esta seria la tabla final con la que trabajariamos


,map.lat,map.lng,Cordenada
1,41.1500,-8.6167,"41.15, -8.6167"
19,1.2931,103.8558,"1.2931, 103.8558"
22,42.3708,-71.1833,"42.3708, -71.1833"
23,35.6850,139.7514,"35.685, 139.7514"
24,-22.9000,-47.0833,"-22.9, -47.0833"
...,...,...,...
165271,53.3331,-6.2489,"53.3331, -6.2489"
165273,33.5000,36.3000,"33.5, 36.3"
165274,14.2667,121.0514,"14.2667, 121.0514"
165275,12.9670,77.5873,"12.967, 77.5873"


In [18]:
#Creamos 
lista = []

In [19]:

# Definir una caché con un tamaño máximo de 1000 elementos
cache = LRUCache(maxsize=1000)


In [20]:
#Esta funcion nos va a permitir obtener el codigo pais a partir de los valores de latitud
#y longitud usando la librerio de geopy, entre otras

def obtener_codigo_pais(cord):
    
    for fila in tqdm(cord.iterrows(), total=len(cord)):
        cordenada = fila[1][0]
        indice = fila[0]
        print(cordenada,indice)
        # Verificar si el resultado ya está en caché
        if cordenada in cache:
            Code = cache[cordenada]
        else:
            try:
                ubi = geoloc.reverse(cordenada, timeout=5)
                if 'address' in ubi.raw and 'country_code' in ubi.raw['address']:
                    Code = ubi.raw['address']['country_code']
                    # Agregar el resultado a la caché
                    cache[cordenada] = Code
                else:
                    Code = 0
            except GeocoderServiceError as e:
                print("Error al geocodificar:", e)
                Code = 0
        
        lista.append([indice, Code])
        time.sleep(1)
    
    return lista

In [23]:
cord_total = cord[['Cordenada']]

In [24]:
cord_total

,Cordenada
1,"41.15, -8.6167"
19,"1.2931, 103.8558"
22,"42.3708, -71.1833"
23,"35.685, 139.7514"
24,"-22.9, -47.0833"
...,...
165271,"53.3331, -6.2489"
165273,"33.5, 36.3"
165274,"14.2667, 121.0514"
165275,"12.967, 77.5873"


In [ ]:
obtener_codigo_pais(cord_total)
#El codigo funciona pero debido a que tarda demaciadas horas, un dia o mas, vamos a 
#precindir de estos datos.

#Pero una ves obtenidos los datos en la lista era cuestion de hacer un marge uniendolos
# a partir del id